In [1]:
from pyspark.sql import SparkSession

spark = (
    
    SparkSession
            .builder
            .config('spark.jars.packages', 'io.delta:delta-core_2.12:2.1.0')
            .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension')
            .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')
            .getOrCreate()
            
)

22/09/23 15:56:18 WARN Utils: Your hostname, desktop-pro resolves to a loopback address: 127.0.1.1; using 192.168.98.179 instead (on interface eth0)
22/09/23 15:56:18 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/carlosbarbosa/.local/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/carlosbarbosa/.ivy2/cache
The jars for the packages stored in: /home/carlosbarbosa/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-82c7e531-c8c2-46b8-9728-ea0a6b9f736d;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.1.0 in central
	found io.delta#delta-storage;2.1.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
	found org.codehaus.jackson#jackson-core-asl;1.9.13 in central
:: resolution report :: resolve 95ms :: artifacts dl 4ms
	:: modules in use:
	io.delta#delta-core_2.12;2.1.0 from central in [default]
	io.delta#delta-storage;2.1.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	org.codehaus.jackson#jackson-core-asl;1.9.13 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| searc

In [3]:
sc = spark.sparkContext

In [2]:
advices = (
    spark
    .read
    .format('delta')
    .load('../datalake/silver/advices_pt/')
)

In [8]:
import pyspark.sql.functions as F

In [10]:
agg_advices = (
    advices
    .withColumn('word', F.explode(F.split(F.col('conselho'), ' ')))
    .groupBy('word')
    .count()
    .sort('count', ascending=False)
)

In [11]:
agg_advices.show()

+--------+-----+
|    word|count|
+--------+-----+
|      de|    8|
|      um|    7|
|     que|    6|
|    você|    5|
|      se|    5|
|      as|    4|
|      em|    4|
|     não|    3|
|problema|    3|
|    mais|    3|
|       é|    3|
|  sempre|    3|
|  coisas|    2|
|   menos|    2|
| melhor.|    2|
|     uma|    2|
|      Se|    2|
|      Um|    2|
| pessoas|    2|
| estiver|    2|
+--------+-----+
only showing top 20 rows



In [12]:
(
    agg_advices
    .write
    .format('delta')
    .mode('overwrite')
    .save('../datalake/gold/agg_advices')
)